In [2]:
using JuMP, Gurobi, CSV, LinearAlgebra
# UW has access to a  fleet of 30 robots
num_robot = 90
# a robot can carry up to 20 pounds of food.
carry_up = 30
# one time the delivery fee is 1.99
delivery_fee = 1.99
# robots move at pedestrian speed
#robots_ speed = 4
# total carry
R1 = CSV.read("data_fourLake.csv")
(p,q) = size(R1)
time = CSV.read("dist_four.csv")
d = 0
cost = 5
time_total = CSV.read("dist_four_final.csv")
period = [1 2 0 0 2 0 1 1 2 0 2 1 2 2 3]
R1

,Adams,Barn&ChadBradley,Column3,Cole&Sull,Dav&Sell,Dejo&Phi,Dejo&Phi_1,Kronshage
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,3,0,0,2,0,2,3
2,2,6,0,0,3,0,3,5
3,4,12,0,0,5,0,5,10
4,7,20,0,0,8,0,9,17
5,9,25,0,0,11,0,12,21
6,9,25,0,0,11,0,12,21
7,7,20,0,0,8,0,9,17
8,4,12,0,0,5,0,6,10
9,2,6,0,0,3,0,3,5


In [12]:
using JuMP, Gurobi
m = Model(with_optimizer(Gurobi.Optimizer,OutputFlag=0))

# m = residence hall number;
# n = time period;
lamda = 0.0000001
# 在每个时间段向不同宿舍派送的机器数量
@variable(m, x[1:(p+d),1:q]>=0,Int)
# 在每个时间段，机器成功从不同宿舍返回的数量
@variable(m, y[1:(p+d),1:q]>=0,Int)
@variable(m, cur[1:(p+d)]>=0, Int)
@variable(m, order[1:(p+d), 1:q] >= 0, Int)
# number of robots assigned to the day, and binary variable indicating when the robots can stop working 
@variable(m, robot, Int)
@variable(m, order_bin[1:(p+d)], Bin) # 0 when current period has orders left, 1 when not# 多少个cycle可以乘以cost

# all the orders must be delivered
//@constraint(m, sum(x[i, j] for i =1:(p+d), j = 1:q) == sum(R1[i,j] for i = 1:p, j = 1:q))
for i = 1:(p+d)
    #for j = 1:q
 //   # 没一个时间段可利用的车辆<=robot总数
             @constraint(m, cur[i] <= robot)
 //   #i时间段所有派单量<=现在可用车辆
             @constraint(m,sum(x[i,:]) <= cur[i]); # 宿舍订单限制
             #@constraint(m,sum(x[l,k] for l = 1:i, k = 1:q))
 //            @constraint(m,sum(x[i,k] for k = 1:q) <= cur[i])
    #i时间段所有派单量
 //            @constraint(m,sum(y[i,k] for k = 1:q) <= cur[i])
    #end
end

for j = 1:q
    #一个宿舍的单必须送完
  // @constraint(m, sum(x[:,j]) == sum(R1[:,j])) 
end
for i = 1:(p+d-1)#对于每个时间段来说
     # when the number of orders left = 0, bin = 1; orders left = 1, bin = 0
   //  @constraint(m, order_bin[i]*sum(order[i,:]) == 0)
    # 下一个时间段可用的车辆=上一个时间段可用的车辆-上一个时间段派出的车辆+回来的车辆     @constraint(m, cur[i+1] == cur[i] + sum(y[i+1, j] for j = 1:q) - sum(x[i,j] for j = 1:q)) # set current available robots
     for j = 1:q
    //    #送出的车辆<=订单数量
        @constraint(m, x[i,j] <= order[i,j])
        if i < p
             #number of orders now can only use the cars that are back
            #下一时刻订单数量=这一时刻没送出的单+R1（运营时间内）
     //        @constraint(m, order[i+1,j] == order[i, j] - x[i, j] + R1[i+1,j]) 
        else 
         //   # 超出运营时间，即为这一时刻没送出的单
             @constraint(m, order[i+1,j] == order[i, j] - x[i, j])
        end
    end
end


for l = 1:q
 //     #初始化订单数量
      @constraint(m, order[1,l] == R1[1,l])
end


# 初始化cur
//@constraint(m, cur[1] == robot)
# constraint to make the robots be back at the right time
for i = 1:(p+d-1) 
      for j=1:q
        #初始化，刚开始回来的车为0
         //   @constraint(m, y[1 ,j] == 0)
            if (period[j] > 0)
                  #@constraint(m, y[i,j]*ind[i,j] == 0) # when y is not 0, ind is 0, there are carts back
                  if (period[j] >= i)   
                //        @constraint(m, y[i,j] == 0)   # the first several cycles are definitely 0    
                  else
                        # for the case that start getting carts back from that dorm
                //        @constraint(m, y[i,j] == x[(i-period[j]),j]) # when the cart is back
                  end
            else
                  @constraint(m, y[i,j] == 0)
            end
            #如果送单时间足够来回，则
            # if (2*time[1,i]<=num_robot)
            #@constraint(M,y[i,j]==x[i,j] for i = 1:n))
            #else
            #如果没送到，回来的车是0
            #@constraint(M,y[i,j]==0 for i = 1:n))
            #end
      end
end

@objective(m, Min, sum(robot*cost*30*sum((p+d-order_bin[l]) for l = 1:(p+d)) for i = 1:(p+d)) + 
                lamda*(sum((x[i,j]*i*30 + time[j,1]) for i = 1:(p+d), j = 1:15) - sum(R1[i,j]*i*30 for i = 1:p, j = 1:q)))
optimize!(m)
value.(cur)
result = value.(x)
#value.(order)
value.(robot)
#value(robot)

Academic license - for non-commercial use only
Academic license - for non-commercial use only


1216.0